In [ ]:
import re
import os
import csv
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from functools import reduce

In [ ]:
def get_data(path):
    dir_path = path
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            with open(os.path.join(dir_path, filename)) as csvfile:
                data = list(csv.reader(csvfile))
            yield data

def create_dataframe(data):
    for d in data:
        try:
            df = pd.DataFrame(d[1:], columns=d[0])
            yield df
        except Exception as e:
            print(f"Error creating DataFrame: {e}")

data = get_data('data/')
df_list = list(create_dataframe(data))
df = reduce(lambda df1,df2: pd.merge(df1,df2,how ='outer'), df_list)
df = df.iloc[:, 5:]

In [ ]:
def clean_space(df, columns: dict):
    df_copy = df.copy()
    for column in columns:
            df_copy[column] = df_copy[column].astype(str)
            df_copy[column] = df_copy[column].str.replace('\n', '').str.strip()
            df_copy[column] = df_copy[column].apply(lambda x: re.sub(r'\s{2,}', columns[column], x))
            yield column, df_copy[column]

def replace_column_data(df, columns_data: dict):
    df_replaced = df.copy()
    for key in columns_data.keys():
        df_replaced[key] = columns_data[key]
    return df_replaced

clean_columns = dict(clean_space(df, {"facility": ',', "space": ' '}))
df = replace_column_data(df, clean_columns)

In [ ]:
df = df.replace(["null", "ไม่มีข้อมูล"], np.NaN)
df["price"] = df["price"].replace(r",", '', regex = True).astype(float)
df["bedroom"] = df["bedroom"].replace("สตูดิโอ", 0).astype(float)
df["bathroom"] = df["bathroom"].astype(float)
df["space"] = df["space"].str.extract(r"([\d\.]+[\d+])").astype(float)
df["year"] = df["year"].str.extract(r"(\d{4})").astype(float)

In [ ]:
combined_list = sum(df["facility"].astype(str).str.split(',') , [])
set(combined_list)